<a href="https://colab.research.google.com/github/MartaCampagnoli/HateSpeechDetection/blob/main/No%20Output%20Notebooks/Classification_Part2_MultiClass_NoOutput.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gensim
import io
import pandas as pd
import numpy as np
import re
import string
import nltk
import gensim.downloader
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from google.colab import files
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.preprocessing import text, sequence
from keras import utils
from keras import models, layers, callbacks, regularizers
from keras.models import Model, Sequential, load_model
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score, zero_one_loss, confusion_matrix, classification_report, ConfusionMatrixDisplay
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

!pip install wget

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
stemmer = nltk.SnowballStemmer("english")

In [ ]:
uploaded = files.upload() #sixcat.csv

In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['sixcat.csv'])) #sixcat

In [ ]:
train, test = train_test_split(df ,test_size=0.30, random_state = 42)

In [ ]:
#modify stopwords list and preprocess data
stopwords = nltk.corpus.stopwords.words('english')
exceptions = ["no", "not" , "don't", "they", "them"]
stop_words = [word for word in stopwords if word not in exceptions]


def preprocess(text):
    text = ''.join([word for word in text])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [stemmer.stem(w) for w in tokens]
    return ' '.join(tokens)

In [ ]:
#Training plots
legend_size = 14

def performance_plot(history):
    plt.figure(figsize=(20,8))

    #Loss
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('loss', size = 12)
    plt.xlabel('epoch', size = 12)
    plt.legend(['train','val'], fontsize = legend_size)
    plt.grid()

    #Accuracy
    plt.subplot(1,2,2)
    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.ylabel('accuracy', size = 12)
    plt.xlabel('epoch', size = 12)
    plt.legend(['train','val'], fontsize = legend_size)
    plt.grid()

    plt.show()

earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=10, restore_best_weights=True)
loss = keras.losses.CategoricalCrossentropy()

# Defining the network structures

In [ ]:
def evaluatemodel(model):
  model.evaluate(X_test300, y_test)
  y_pred = model.predict(X_test300)
  y_pred = np.argmax(y_pred, axis=1)
  return  y_pred

In [ ]:
#cnn
def deepcnn(vocab_size, embedding_matrix, num_class):
  model = Sequential()
  model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable=False))
  model.add(Conv1D(64, 3, padding="same", activation='relu'))
  model.add(Conv1D(64, 3, padding="same", activation='relu'))
  model.add(Conv1D(64, 3, padding="same", activation='relu'))
  model.add(MaxPooling1D(3, padding='same'))
  model.add(Dropout(0.5))
  model.add(Conv1D(128, 3, padding="same", activation='relu'))
  model.add(Conv1D(128, 3, padding="same", activation='relu'))
  model.add(Conv1D(128, 3, padding="same", activation='relu'))
  model.add(MaxPooling1D(3, padding='same'))
  model.add(Dropout(0.4))
  model.add(Conv1D(64, 3, padding="same", activation='relu'))
  model.add(Conv1D(64, 3, padding="same", activation='relu'))
  model.add(Conv1D(64, 3, padding="same", activation='relu'))
  model.add(MaxPooling1D(3, padding='same'))
  model.add(Dropout(0.3))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(num_class, activation = 'softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
  return model

#bilstm

def bidlstm(vocab_size, embedding_matrix, num_class):
  model = Sequential()
  model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable=False))
  model.add(Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)))
  model.add(Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)))
  model.add(Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)))
  model.add(Dropout(0.4))
  model.add(Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)))
  model.add(Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)))
  model.add(Bidirectional(tf.keras.layers.LSTM(32)))
  model.add(Dropout(0.2))
  model.add(Dense(num_class, activation = 'softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
  return model

# Pretrained embeddings: CNN and Bidirectional


In [ ]:
#setting training and test
X_train, X_test, train_target, test_target = train['text'], test['text'], train['target'], test['target']

In [ ]:
X_train = X_train.apply(preprocess)
X_test = X_test.apply(preprocess)

In [ ]:
#encoding the labels and saving the number of classes as a variable
encoder = LabelEncoder()
encoder.fit(train_target)
y_train = encoder.transform(train_target)
y_test = encoder.transform(test_target)
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [ ]:
#tokenization and padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1
max_length = 300
X_train300 = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test300 = pad_sequences(X_test, maxlen=max_length, padding='post')

In [ ]:
#function to try for pretrained models
def embedpretr(model, token):
  embedding_matrix = np.zeros((vocab_size, 300))
  for word, i in token.word_index.items():
    if word in model:
        embedding_matrix[i] = model[word]
  return embedding_matrix

In [ ]:
#Available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))

In [ ]:
w2vvectors = gensim.downloader.load('word2vec-google-news-300')
fast = gensim.downloader.load('fasttext-wiki-news-subwords-300')

In [ ]:
embedding_matrix_pre = embedpretr(w2vvectors, tokenizer)

In [ ]:
#cnn with pretrained w2c
pretrainedcnnw = deepcnn(vocab_size, embedding_matrix_pre, num_classes)
history = pretrainedcnnw.fit(X_train300, y_train, epochs=50, batch_size=128, verbose = 0, validation_split=0.1, callbacks=[earlystopping])
performance_plot(history)

In [ ]:
#evaluating the cnn
predictions = evaluatemodel(pretrainedcnnw)

In [ ]:
print(metrics.classification_report(test_target, predictions))
ConfusionMatrixDisplay.from_predictions(test_target, predictions, cmap = 'summer')

In [ ]:
pretrainedlstmw = bidlstm(vocab_size, embedding_matrix_pre, num_classes)
history = pretrainedlstmw.fit(X_train300, y_train, epochs=50, batch_size=32, verbose = 0, validation_split=0.1, callbacks=[earlystopping])
performance_plot(history)

In [ ]:
predictions = evaluatemodel(pretrainedlstmw)

In [ ]:
print(metrics.classification_report(test_target, predictions))
ConfusionMatrixDisplay.from_predictions(test_target, predictions, cmap = 'summer')

In [ ]:
#fasttext

In [ ]:
embedding_matrix_pref = embedpretr(fast, tokenizer)

In [ ]:
pretrainedcnnf = deepcnn(vocab_size, embedding_matrix_pref, num_classes)
history = pretrainedcnnf.fit(X_train300, y_train, epochs=50, batch_size=128, verbose = 0, validation_split=0.1, callbacks=[earlystopping])
performance_plot(history)

In [ ]:
predictions = evaluatemodel(pretrainedcnnf)

In [ ]:
print(metrics.classification_report(test_target, predictions))
ConfusionMatrixDisplay.from_predictions(test_target, predictions, cmap = 'summer')

In [ ]:
pretrainedlstmf = bidlstm(vocab_size, embedding_matrix_pref, num_classes)
history = pretrainedlstmf.fit(X_train300, y_train, epochs=50, batch_size=32, verbose = 0, validation_split=0.1, callbacks=[earlystopping])
performance_plot(history)

In [ ]:
predictions = evaluatemodel(pretrainedlstmf)

In [ ]:
print(metrics.classification_report(test_target, predictions))
ConfusionMatrixDisplay.from_predictions(test_target, predictions, cmap = 'summer')